In [21]:
import numpy as np
import random
from pathlib import Path

from limlam_mocker import limlam_mocker as llm
import lnn as lnn

In [58]:
def make_random_map(catalogs, haloLoc, mapLoc):
    catalog = random.choice(catalogs)
    
    return(make_random_map_from_cat(catalog, haloLoc, mapLoc))

def make_random_map_from_cat(catalog, haloLoc, mapLoc):
    ### make a random paramDict
    paramDict = {}
    paramDict = make_paramDict(paramDict=paramDict)
    
    ### make sure the map directory exists
    lnn.checkDirectoryPath(mapLoc)
    
    param = lnn.getParams(haloLoc + catalog, mapLoc + catalog, **paramDict)
    
    file_name = coeffs_to_file_name(param.model, param.coeffs, mapLoc, catalog)
    param.map_output_file = file_name
    return(param.map_output_file)
#     print(param.halo_catalogue_file, '\n', param.map_output_file, '\n', param.model, '\n', param.coeffs)
#     lnn.makeAndSaveMapAndLumFunc(param, verbose=False)

def make_paramDict(paramDict, model=None, default=False):
    model_list = ['Li', 'Padmanabhan', 'Breysse']
    means = []
    sig = []
    
    ### randomly pick a model if none is given
    if model == None:
        model = random.choice(model_list)
    
    ### set up the coefficients for the model
    if model == 'Li':
        ### parameter info from arxiv:1503.08833 
        ### log_delta_mf, alpha, beta, sigma_sfr (>0), sigma_lc0 (>0)
        means = [0.0, 1.37,-1.74, 0.3, 0.3]
        ### loose priors
        sig = [0.3, 0.37, 3.74, 0.1, 0.1]
        ### stronger priors
        #sig = [0.3, 0.04, 0.4, 0.1, 0.1]
    elif model == 'Padmanabhan':
        ### parameter info from arxiv:1706.01471 
        ### m10, m11, n10, n11, b10, b11, y10, y11
        means = [4.17e12, -1.17, 0.0033, 0.04, 0.95, 0.48, 0.66, -0.33]
        ### priors
        sig = [2.03e12, 0.85, 0.0016, 0.03, 0.46, 0.35, 0.32, 0.24]
    elif model == 'Breysse':
        ### parameter info from arxiv:1706.01471 
        ### A, b
        means = [2e-6, 1.0]
        ### priors
        sig = [5e-7, 0.125]
    else:
        sys.exit('\n\n\tYour model, '+model+', does not seem to exist\n\t\tPlease check src/halos_to_luminosity.py in limlam_mocker to add it\n\n')

    ### set the model parameter for limlam
    paramDict['model'] = model
    
    ### if default is set, use the default values
    if default:
        coeffs = means
    ### if default is not set, find a random realization of the given model's parameters
    else:
        coeffs = [np.random.normal(x,y) for x,y in zip(means, sig)]    
        
        ### make sure last two parameters in the Li model are non-negative
        if model == 'Li':
            for i in [-1,-2]:
                if coeffs[i] < 0:
                    coeffs[i] = 0
        
    ### set coeffs parameter for limlam
    paramDict['coeffs'] = coeffs
    
    return(paramDict)

def coeffs_to_file_name(model, coeffs, mapLoc, catalog):
    file_name = '__' + model + '_'

    for coeff in coeffs:
        file_name += '{:.3e}_'.format(coeff)

    full_file_name = mapLoc + catalog[:-4] + file_name[:-1] + catalog[-4:]
        
    return(full_file_name)

def path_to_coeffs(path):
    file_name = path.split('/')[-1]
    model, coeffs = file_name_to_coeffs(file_name)
    
    return(model, coeffs)

def file_name_to_coeffs(file_name):
    cat, lum_info = file_name.split('__')

    lum_list = lum_info[:-4].split('_')
    model = lum_list[0]
    coeffs = list(map(float, lum_list[1:]))
    
    return(model, coeffs)

In [3]:
### set the location of the halo catalogs and maps
haloLoc = '../catalogues2/'
mapLoc = '../maps2/test2/'

In [4]:
### get the subfield names
subFields = lnn.loadSubFields(haloLoc)

In [52]:
map_names = []

for i in range(10000):
    temp_map = make_random_map(subFields, haloLoc, mapLoc)
    map_names.append(temp_map)

In [11]:
len(map_names)

10000

In [15]:
len(set(map_names))

9999

In [14]:
map_names[0] = map_names[1]

In [53]:
map_names[0]

'../maps2/test2/COMAP_z2.39-3.44_1140Mpc_seed_13765_rotate_0_subfield_18__Li_-1.280e-01_2.056e+00_-3.450e+00_3.376e-01_5.537e-01.npz'

In [43]:
test_name = map_names[6]

In [37]:
file_name = test_name.split('/')[-1]
cat, lum_info = file_name.split('__')

In [22]:
file_name = Path(test_name)

In [32]:
for p in file_name.parents:
    print(p)

../maps2/test2
../maps2
..
.


In [33]:
file_name.name

'COMAP_z2.39-3.44_1140Mpc_seed_13669_rotate_0_subfield_8__Breysse_1.458e-061.061e+00.npz'

In [38]:
file_name

'COMAP_z2.39-3.44_1140Mpc_seed_13669_rotate_0_subfield_8__Breysse_1.458e-061.061e+00.npz'

In [57]:
test_name = map_names[6]
file_name = test_name.split('/')[-1]
cat, lum_info = file_name.split('__')

lum_list = lum_info[:-4].split('_')
model = lum_list[0]
coeffs = list(map(float, lum_list[1:]))


COMAP_z2.39-3.44_1140Mpc_seed_13719_rotate_0_subfield_34 
 Padmanabhan_4.815e+12_-2.070e+00_1.660e-03_7.000e-02_6.221e-01_4.946e-01_7.395e-01_-4.002e-01.npz
Padmanabhan [4815000000000.0, -2.07, 0.00166, 0.07, 0.6221, 0.4946, 0.7395, -0.4002]


In [61]:
path_to_coeffs(map_names[8])

('Li', [0.07593, 1.449, -4.164, 0.3646, 0.4365])